## Redes neuronales

In [1]:
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#### Funciones de activación

In [2]:
def sigmoid(x):
    return 1.0/(1.0 + np.exp(-x))

def sigmoid_derivada(x):
    return sigmoid(x)*(1.0-sigmoid(x))


def tanh(x):
    return np.tanh(x)

def tanh_derivada(x):
    return 1.0 - x**2



#### Clase para Red Neuronal

In [3]:
class NeuralNetwork:
 
        # Recibe arreglo de neuronas en las capas y función de activación
    def __init__(self, layers, f_activation):

        
        if f_activation == 'sigmoid':
            self.f_activation = sigmoid
            self.f_activation_prime = sigmoid_derivada
        elif f_activation == 'tanh':
            self.f_activation = tanh
            self.f_activation_prime = tanh_derivada
            
        # declarar pesos
        self.weights = []
        # declarar errores
        self.deltas = []
        
        
        # asignación de valores aleatorios a las neuronas de la capa de entrada y la capa oculta 
        
        for i in range(1, len(layers) - 1):     
            r = 2*np.random.random((layers[i-1] + 1, layers[i] + 1)) - 1   #valores [-1,1]
            self.weights.append(r)
            
        print("Input random weights \n ",r)
            
        # asignación de valores aleatorios a la capa de salida
        r = 2*np.random.random( (layers[i] + 1, layers[i+1])) - 1  #valores [-1,1]
        self.weights.append(r)
        
        print("Output random weights \n ",r)
        
        

    #Función para entrenamiento
    def fit(self, X, y, learning_rate=0.2, iterations=100):

        # Agregar columna extra con unos para obtener la variable B0 (Bias, ordenada al origen)
        ones = np.atleast_2d(np.ones(X.shape[0]))
        X = np.concatenate((ones.T, X), axis=1)
        a = []
        
        # Ciclo para el número total de iteraciones
        for k in range(iterations):
            i = np.random.randint(X.shape[0])
            a = [X[i]]

            for l in range(len(self.weights)):
                    dot_value = np.dot(a[l], self.weights[l])
                    activation = self.f_activation(dot_value)
                    a.append(activation)
            # Calculo de la diferencia en la capa de salida y el valor obtenido
            error = y[i] - a[-1]
            
            # backpropagation
            # 1. Multiplcar los delta de salida con las activaciones de entrada 
            #    para obtener el gradiente del peso.
            deltas = [error * self.f_activation_prime(a[-1])]
            
            # Empezamos en el segundo layer hasta el ultimo
            # (Una capa anterior a la de salida)
            for l in range(len(a) - 2, 0, -1): 
                deltas.append(deltas[-1].dot(self.weights[l].T)*self.f_activation_prime(a[l]))
            self.deltas.append(deltas)

            # invertir
            deltas.reverse()

            # 2. actualizar el peso restandole un porcentaje del gradiente
            for i in range(len(self.weights)):
                layer = np.atleast_2d(a[i])
                delta = np.atleast_2d(deltas[i])
                self.weights[i] += learning_rate * layer.T.dot(delta)

            if k % 10000 == 0: print('iterations:', k) 
                
                
    def predict(self, x): 
        ones = np.atleast_2d(np.ones(x.shape[0]))
        a = np.concatenate((np.ones(1).T, np.array(x)), axis=0)
        for l in range(0, len(self.weights)):
            a = self.f_activation(np.dot(a, self.weights[l]))
        return a

    def print_weights(self):
        print("Pesos resultantes")
        for i in range(len(self.weights)):
            print(self.weights[i])


### Lectura y separación del dataset

In [4]:
dataset = pd.read_csv("~/Documents/BM/Curso DS/gender_classification_v7.csv",dtype='str',encoding = "ISO-8859-1")

In [5]:
dataset = pd.read_csv("~/Documents/BM/Curso DS/gender_classification_v7.csv",dtype='str',encoding = "ISO-8859-1")
dataset["long_hair"] = dataset["long_hair"].astype(float)
dataset["forehead_width_cm"] = dataset["forehead_width_cm"].astype(float)
dataset["forehead_height_cm"] = dataset["forehead_height_cm"].astype(float)
dataset["nose_wide"] = dataset["nose_wide"].astype(float)
dataset["nose_long"] = dataset["nose_long"].astype(float)
dataset["lips_thin"] = dataset["lips_thin"].astype(float)
dataset["distance_nose_to_lip_long"] = dataset["distance_nose_to_lip_long"].astype(float)

In [6]:
dataset.describe()

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long
count,5002.000000,5002.000000,5002.000000,5002.000000,5002.000000,5002.000000,5002.000000
mean,0.869652,13.181927,5.946202,0.494002,0.507997,0.493203,0.499000
std,0.336719,1.107462,0.541269,0.500014,0.499986,0.500004,0.500049
min,0.000000,11.400000,5.100000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,12.200000,5.500000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,13.100000,5.900000,0.000000,1.000000,0.000000,0.000000
75%,1.000000,14.000000,6.400000,1.000000,1.000000,1.000000,1.000000
max,1.000000,15.500000,7.100000,1.000000,1.000000,1.000000,1.000000


In [7]:
clean_dataset = dataset.drop_duplicates()

In [8]:
X = clean_dataset.iloc[:,:7]
y = clean_dataset.iloc[:,-1]
y = y.to_frame()
y[y.gender == 'Male'] = -1.0
y[y.gender == 'Female'] = 1.0
y['gender'] = y['gender'].astype(float)

In [9]:
x_train,x_test,y_train,y_test =train_test_split(X.to_numpy(),y.to_numpy(),test_size=0.25)

### Entrenamiento y evaluación

In [10]:
layers = [x_train.shape[1],4,1]

In [11]:
layers

[7, 4, 1]

In [12]:
model = NeuralNetwork(layers, 'tanh' )

Input random weights 
  [[-0.80677773 -0.63287997 -0.09426267 -0.06310499  0.27745043]
 [ 0.50994477 -0.07002992  0.8032736   0.01667369 -0.15597731]
 [ 0.69470291  0.43882478 -0.57454006  0.95024416  0.03883098]
 [ 0.12110981 -0.53180485 -0.6469174   0.54647923 -0.35830332]
 [-0.2882596   0.20368436 -0.5334562  -0.82874237  0.468836  ]
 [-0.28865399 -0.10774107 -0.37750696  0.74824327  0.18261649]
 [ 0.02035047 -0.70676296 -0.82777758  0.36677221  0.93765839]
 [ 0.06521507  0.48936494 -0.83259472 -0.9757354   0.05906962]]
Output random weights 
  [[-0.51465924]
 [ 0.26735173]
 [-0.31704934]
 [ 0.22354084]
 [-0.25494979]]


In [13]:
model.fit(x_train, y_train, learning_rate=0.001,iterations=10000 )

iterations: 0


In [14]:
index=0
output = 1.0
error = 0
for element in x_test:
    if model.predict(element) < 0:
        output = -1.0
    else:
        output = 1.0
    print("X:",element,"y:",y_test[index],"Network:",model.predict(element), "Salida: ", output )
    if y_test[index] != output:
        error += 1
   
    index=index+1
    


X: [ 1.  11.6  5.3  0.   0.   0.   0. ] y: [1.] Network: [0.86757382] Salida:  1.0
X: [ 1.  13.7  5.7  1.   1.   1.   1. ] y: [-1.] Network: [-0.83951469] Salida:  -1.0
X: [ 1.  13.7  6.   1.   1.   1.   1. ] y: [-1.] Network: [-0.83290767] Salida:  -1.0
X: [ 1.  13.2  6.7  0.   1.   0.   1. ] y: [-1.] Network: [0.70273216] Salida:  1.0
X: [ 1.  13.4  6.9  0.   0.   1.   1. ] y: [-1.] Network: [0.54611428] Salida:  1.0
X: [ 0.  14.3  5.2  1.   1.   1.   1. ] y: [-1.] Network: [-0.85631574] Salida:  -1.0
X: [ 0.  11.7  5.4  0.   0.   0.   0. ] y: [1.] Network: [0.86129574] Salida:  1.0
X: [ 1.  13.8  6.2  1.   0.   1.   1. ] y: [-1.] Network: [-0.62406752] Salida:  -1.0
X: [ 1.  11.9  5.8  0.   1.   0.   0. ] y: [1.] Network: [0.82053719] Salida:  1.0
X: [ 1.  12.4  6.1  1.   0.   0.   0. ] y: [1.] Network: [0.79331788] Salida:  1.0
X: [ 1.  14.2  7.   1.   1.   1.   0. ] y: [-1.] Network: [-0.44209828] Salida:  -1.0
X: [ 0.  11.8  5.1  1.   1.   1.   0. ] y: [-1.] Network: [-0.7967513]

X: [ 0.  12.8  5.5  0.   0.   0.   0. ] y: [1.] Network: [0.86572274] Salida:  1.0
X: [ 1.  13.3  5.3  0.   1.   1.   1. ] y: [-1.] Network: [-0.61149973] Salida:  -1.0
X: [ 1.  12.2  6.2  0.   0.   0.   1. ] y: [1.] Network: [0.83120628] Salida:  1.0
X: [ 0.  11.8  6.5  1.   0.   1.   1. ] y: [1.] Network: [-0.71900978] Salida:  -1.0
X: [ 1.  12.1  6.9  1.   1.   0.   1. ] y: [-1.] Network: [-0.27262552] Salida:  -1.0
X: [ 1.  11.6  6.2  1.   1.   0.   1. ] y: [-1.] Network: [-0.46982316] Salida:  -1.0
X: [ 1.  12.2  6.1  1.   1.   1.   1. ] y: [-1.] Network: [-0.8393668] Salida:  -1.0
X: [ 1.  13.3  6.8  1.   1.   0.   0. ] y: [-1.] Network: [0.59939832] Salida:  1.0
X: [ 1.  12.9  6.3  0.   0.   0.   1. ] y: [1.] Network: [0.83812633] Salida:  1.0
X: [ 1.  14.4  7.   1.   1.   1.   1. ] y: [-1.] Network: [-0.79492251] Salida:  -1.0
X: [ 1.  13.1  6.8  1.   1.   1.   1. ] y: [-1.] Network: [-0.8147902] Salida:  -1.0
X: [ 1.  15.2  6.5  1.   1.   0.   1. ] y: [-1.] Network: [-0.191779

X: [ 1.  15.4  5.7  1.   0.   1.   1. ] y: [-1.] Network: [-0.64375922] Salida:  -1.0
X: [ 0.  13.   5.4  0.   0.   0.   0. ] y: [1.] Network: [0.86516439] Salida:  1.0
X: [ 1.  13.8  5.3  0.   0.   1.   0. ] y: [1.] Network: [0.746222] Salida:  1.0
X: [ 1.  13.3  5.6  1.   0.   1.   1. ] y: [-1.] Network: [-0.70483012] Salida:  -1.0
X: [ 0.  13.   5.1  0.   0.   1.   1. ] y: [-1.] Network: [-0.14694433] Salida:  -1.0
X: [ 1.  11.5  5.1  0.   0.   0.   0. ] y: [1.] Network: [0.86537344] Salida:  1.0
X: [ 1.  14.   5.5  0.   1.   0.   1. ] y: [1.] Network: [0.56812103] Salida:  1.0
X: [ 1.  13.7  5.5  1.   1.   0.   1. ] y: [-1.] Network: [-0.51677782] Salida:  -1.0
X: [ 1.  12.6  5.8  1.   0.   0.   0. ] y: [1.] Network: [0.78020099] Salida:  1.0
X: [ 1.  12.1  6.3  0.   0.   0.   1. ] y: [1.] Network: [0.83327045] Salida:  1.0
X: [ 1.  12.4  6.8  1.   1.   1.   0. ] y: [-1.] Network: [-0.56994161] Salida:  -1.0
X: [ 1.  13.3  6.9  1.   1.   0.   1. ] y: [-1.] Network: [-0.19603537] Sa

X: [ 1.  13.7  5.7  0.   0.   0.   0. ] y: [1.] Network: [0.87497332] Salida:  1.0
X: [ 1.  15.   6.4  1.   1.   1.   1. ] y: [-1.] Network: [-0.81295196] Salida:  -1.0
X: [ 1.  14.1  5.3  1.   1.   1.   1. ] y: [-1.] Network: [-0.84536534] Salida:  -1.0
X: [ 0.  11.9  6.3  1.   1.   0.   1. ] y: [-1.] Network: [-0.5675945] Salida:  -1.0
X: [ 1.  14.6  5.7  1.   1.   0.   1. ] y: [-1.] Network: [-0.43525026] Salida:  -1.0
X: [ 1.  14.1  5.6  1.   1.   1.   1. ] y: [-1.] Network: [-0.83969202] Salida:  -1.0
X: [ 1.  15.5  5.7  1.   1.   1.   0. ] y: [-1.] Network: [-0.62074616] Salida:  -1.0
X: [ 1.  13.6  5.1  0.   1.   0.   0. ] y: [1.] Network: [0.80982189] Salida:  1.0
X: [ 1.  14.2  6.3  0.   0.   1.   0. ] y: [1.] Network: [0.80702701] Salida:  1.0
X: [ 1.  12.4  5.1  1.   1.   1.   1. ] y: [-1.] Network: [-0.8544326] Salida:  -1.0
X: [ 1.  14.6  5.4  0.   1.   1.   0. ] y: [-1.] Network: [0.30309228] Salida:  1.0
X: [ 1.  14.3  5.2  0.   0.   0.   0. ] y: [1.] Network: [0.872463]

X: [ 1.  11.9  6.   0.   0.   1.   0. ] y: [1.] Network: [0.75902649] Salida:  1.0
X: [ 1.  12.6  6.5  0.   0.   0.   0. ] y: [1.] Network: [0.87738742] Salida:  1.0
X: [ 1.  11.5  5.7  1.   1.   1.   1. ] y: [-1.] Network: [-0.85012735] Salida:  -1.0
X: [ 1.  13.4  6.1  0.   0.   0.   0. ] y: [1.] Network: [0.87682876] Salida:  1.0
X: [ 1.  12.8  6.1  1.   1.   1.   1. ] y: [-1.] Network: [-0.83573727] Salida:  -1.0
X: [ 0.  13.2  7.1  1.   1.   1.   1. ] y: [-1.] Network: [-0.82630229] Salida:  -1.0
X: [ 1.  13.   6.1  1.   0.   0.   0. ] y: [1.] Network: [0.7997955] Salida:  1.0
X: [ 1.  11.7  6.   1.   1.   0.   0. ] y: [1.] Network: [0.36567067] Salida:  1.0
X: [ 1.  15.3  5.2  1.   1.   1.   0. ] y: [-1.] Network: [-0.68613303] Salida:  -1.0
X: [ 0.  14.2  5.6  1.   1.   1.   1. ] y: [-1.] Network: [-0.85130098] Salida:  -1.0
X: [ 1.  14.9  6.3  1.   1.   1.   0. ] y: [-1.] Network: [-0.54787156] Salida:  -1.0
X: [ 1.  13.2  5.3  0.   0.   0.   0. ] y: [1.] Network: [0.87126469] 

X: [ 1.  14.9  5.5  1.   1.   1.   1. ] y: [-1.] Network: [-0.83807028] Salida:  -1.0
X: [ 1.  13.9  5.8  1.   1.   1.   0. ] y: [-1.] Network: [-0.6536711] Salida:  -1.0
X: [ 1.  14.   5.9  0.   0.   1.   0. ] y: [1.] Network: [0.78651779] Salida:  1.0
X: [ 1.  11.9  6.4  0.   0.   1.   0. ] y: [1.] Network: [0.77911958] Salida:  1.0
X: [ 1.  14.6  7.1  1.   1.   1.   1. ] y: [-1.] Network: [-0.78825915] Salida:  -1.0
X: [ 1.  12.7  6.4  1.   1.   1.   1. ] y: [-1.] Network: [-0.82938232] Salida:  -1.0
X: [ 1.  14.2  5.8  0.   0.   0.   0. ] y: [1.] Network: [0.87629067] Salida:  1.0
X: [ 1.  14.5  6.3  0.   1.   1.   1. ] y: [-1.] Network: [-0.36570371] Salida:  -1.0
X: [ 0.  13.4  5.3  1.   1.   1.   1. ] y: [-1.] Network: [-0.85755397] Salida:  -1.0
X: [ 1.  14.2  5.3  1.   1.   1.   0. ] y: [-1.] Network: [-0.70025133] Salida:  -1.0
X: [ 1.  12.4  6.3  1.   0.   0.   0. ] y: [1.] Network: [0.80238685] Salida:  1.0
X: [ 1.  13.9  6.3  1.   1.   0.   0. ] y: [-1.] Network: [0.538205

In [15]:
accuracy =  1 - (error / index)
print("errores:", error)
print("Observaciones: ", index)
print("Precisión: ", accuracy)

errores: 56
Observaciones:  809
Precisión:  0.930778739184178


In [16]:
model.print_weights()

Pesos resultantes
[[-0.80677775 -0.62685669 -0.09426267 -0.06310499  0.2021997 ]
 [ 0.50994477 -0.06614458  0.8032736   0.01667369 -0.15412982]
 [ 0.69470269  0.50269388 -0.57454007  0.95024416 -0.04837388]
 [ 0.12110974 -0.51784648 -0.6469174   0.54647923 -0.22342576]
 [-0.28825956  0.17874169 -0.5334562  -0.82874237  0.91578478]
 [-0.28865395 -0.12873182 -0.37750696  0.74824327  0.67027335]
 [ 0.0203505  -0.71388129 -0.82777758  0.36677221  0.97859311]
 [ 0.06521512  0.46160702 -0.83259472 -0.9757354   0.70450381]]
[[-0.57947431]
 [ 0.1845331 ]
 [-0.25223429]
 [ 0.15872579]
 [-1.4012098 ]]


### Entrenamiento y evaluación algoritmo de SKLEARN

In [17]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [39]:
model2 = MLPClassifier(activation = 'tanh', alpha=0.001, solver = 'lbfgs',  
                       hidden_layer_sizes=(4, 1), max_iter = 10000)

model2.fit(x_train, y_train)


/Users/cecilia/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


MLPClassifier(activation='tanh', alpha=0.001, hidden_layer_sizes=(4, 1),
              max_iter=10000, solver='lbfgs')

In [40]:
y_true = model2.predict(x_test)

In [41]:
accuracy2 = accuracy_score(pd.DataFrame(y_true), y_test)
print("Precisión: ",accuracy2)

Precisión:  0.9616810877626699


https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html